In [1]:
import requests
from datetime import datetime, timedelta
from pathlib import Path

In [14]:
headers = {
    "Accept": "text/html,application/xhtml+xml,application/xml;q=0.9,image/avif,image/webp,image/apng,*/*;q=0.8,application/signed-exchange;v=b3;q=0.7",
    "Accept-Encoding": "gzip, deflate, br, zstd",
    "Accept-Language": "en-GB,en-US;q=0.9,en;q=0.8",
    "Connection": "keep-alive",
    # Keep a default Cookie here. Individual calls may override it by passing `cookie`.
    "Cookie": "_gid=GA1.2.259970603.1761098063; hd_user_https=c8852d319c83724285e92fba54c60d1a3bd74063c57493d4ce49b692f6926883:1761101811544; hd_user_id_https=09e1b4652c9f2197027d4a49d9013e48d53e8c1f9abfc09c269c45914a7f1e87e65560dc0b9cac0d2c1cfd30aeb217e43c009870cec9660d65a4fbfc0703b1f7; hd_device_https=454a1edd34bd77111b1f0a28714077e0f7000920c64a2556802086f3dc2c0842:1fe3b6517d503b4c141733bd679f237f2cacd0674db5681fdd1aa0234c640016; __hd_fingerprint=1847501811636; _ga_9RGBXZFS3X=GS2.1.s1761101812$o2$g1$t1761101831$j41$l0$h0; _ga=GA1.1.1657724508.1761098060; XSRF-TOKEN=eyJpdiI6ImlHODkyYnNWR01TeTJRUTlNYjlqc0E9PSIsInZhbHVlIjoiQ0ZXTHlNOGxiSjJGL0VoZFlibU9QRnpMVkcwVUxTWFJaaW0zRDlNZ3dDU2hTMm5HRDVKNyt4RytZTVBkV25KR00xREFtNkxKSGlSS0s1QmxBOE81aWREWGNIbURxMGJzVnlDa1JPS2oraHJjd0hoK3ByczJxRTlKRXd2TnpHTHUiLCJtYWMiOiI2NGIzYWJjNWMyODg2N2Q5M2MyYTZhNzQ3NDQ4YmVlMzlkZTg0MDkxZjY5MDBiMDY2YzJiYmZhZGQxNDViNzg3IiwidGFnIjoiIn0%3D; ncdex_session=eyJpdiI6InRucVhDbWNDUDYrbXZXdVM2M1pxMlE9PSIsInZhbHVlIjoibXJiMjZQeXBYd2hneUFpRE1iWmMvV1JGZWdDdzB0eS84M1BieVVVZ3dGOHY2TXZCeVRxMm5wSC93RDFiZ2tnRS9aV2dBMnZxTGdYNlo5RTBvSm1YdllIRjBiYWo1eGRJT3RKQnN2T2twRFpZd203cDVaSDFkTEowYVAwWHphK3kiLCJtYWMiOiJiMDgwODA4MGIxYjE5MjZhNzU1ZmYwN2E4MTEwOTU3OGIwYzliYzMzYjEzYjZjMTdkMTFkYWQxNDdmOTI2M2JjIiwidGFnIjoiIn0%3D; _ga_6BC6S1TGL7=GS2.1.s1761101812$o2$g1$t1761102942$j60$l0$h0",
    "Host": "ncdex.com",
    "Referer": "https://ncdex.com/markets/bhavcopy",
    "User-Agent": "Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/139.0.0.0 Safari/537.36",
}

In [3]:
_session = requests.Session()
THROTTLE_SECONDS = 2

In [4]:
url = "https://ncdex.com/markets/bhavcopy?file_type_both=final&type_both=bhavcopy&bhavcopy_for=both&filedate_both=01-Aug-2025&format_both=file"

In [15]:
def get_bhavcopy(date_str: str, save_path: str):
    # Parse input to standard YYYY-MM-DD
    dt = datetime.strptime(date_str, "%d-%b-%Y")
    save_name = dt.strftime("%Y-%m-%d") + ".zip"
    year = dt.strftime("%Y")
    month = dt.strftime("%b")
    
    url = (
        "https://ncdex.com/markets/bhavcopy?"
        f"file_type_both=final&type_both=bhavcopy&bhavcopy_for=both&filedate_both={date_str}&format_both=file"
    )
    
    try:
        folder = Path(save_path) / year / month
        folder.mkdir(parents=True, exist_ok=True)
        full_path = folder / save_name
        
        # Send a GET request to the URL. The 'stream=True' parameter
        # is helpful for large files as it avoids loading the entire
        # file into memory at once.
        print(f"Downloading bhavcopy for {date_str} -> {save_name}")
        print(url)
        with requests.get(url, headers=headers, stream=True) as r:
            r.raise_for_status()
            with open(full_path, "wb") as f:
                for chunk in r.iter_content(chunk_size=8192):
                    if chunk:  # filter out keep-alive
                        f.write(chunk)
    
    except requests.exceptions.RequestException as e:
        print(f"An error occurred: {e}")

In [6]:
def get_bhavcopy_range(start_date: str, end_date: str, save_path: str):
    """
    Iterate over dates between start_date and end_date (inclusive),
    and call download_bhavcopy for each.
    
    Args:
        start_date (str): "YYYY-MM-DD"
        end_date   (str): "YYYY-MM-DD"
        save_path  (str): Base path to save
    """
    start = datetime.strptime(start_date, "%Y-%m-%d")
    end = datetime.strptime(end_date, "%Y-%m-%d")
    
    current = start
    while current <= end:
        date_str = current.strftime("%d-%b-%Y")  # e.g. 01-Aug-2025
        try:
            get_bhavcopy(date_str, save_path)
        except Exception as e:
            print(f"❌ Failed for {date_str}: {e}")
        current += timedelta(days=1)

In [ ]:
get_bhavcopy_range("2025-01-01", "2025-08-18", "../data/ncdex")

In [17]:
get_bhavcopy("12-Aug-2025", "../data/ncdex")

https://ncdex.com/markets/bhavcopy?file_type_both=final&type_both=bhavcopy&bhavcopy_for=both&filedate_both=12-Aug-2025&format_both=file
An error occurred: None
